In [2]:
import os 
os.chdir('../')

In [1]:
import torch 
from classification.classifier import ResNetECG 

ecg_length = 1024 
ecg_channel = 12

x = torch.rand((4, ecg_length, ecg_channel)) 
model_res = ResNetECG(num_classes=10)

In [2]:
model_res(x).size()

torch.Size([4, 10])

In [3]:
for name, param in model_res.named_parameters():
    if "weight" in name:  # Filter only weight matrices
        print(f"Layer: {name} | Matrix size: {param.size()}")

total_params = sum(p.numel() for p in model_res.parameters())
print(f"Total number of parameters in the model: {total_params}")

Layer: resnet.first_conv.conv.weight | Matrix size: torch.Size([64, 12, 16])
Layer: resnet.first_bn.weight | Matrix size: torch.Size([64])
Layer: resnet.stage_list.0.block_list.0.bn1.weight | Matrix size: torch.Size([64])
Layer: resnet.stage_list.0.block_list.0.conv1.conv.weight | Matrix size: torch.Size([64, 64, 1])
Layer: resnet.stage_list.0.block_list.0.bn2.weight | Matrix size: torch.Size([64])
Layer: resnet.stage_list.0.block_list.0.conv2.conv.weight | Matrix size: torch.Size([64, 16, 16])
Layer: resnet.stage_list.0.block_list.0.bn3.weight | Matrix size: torch.Size([64])
Layer: resnet.stage_list.0.block_list.0.conv3.conv.weight | Matrix size: torch.Size([64, 64, 1])
Layer: resnet.stage_list.0.block_list.0.se_fc1.weight | Matrix size: torch.Size([32, 64])
Layer: resnet.stage_list.0.block_list.0.se_fc2.weight | Matrix size: torch.Size([64, 32])
Layer: resnet.stage_list.0.block_list.1.bn1.weight | Matrix size: torch.Size([64])
Layer: resnet.stage_list.0.block_list.1.conv1.conv.weight

In [6]:
from classification.classifier import TransformerECG 

# Example usage
embed_size = 256
num_heads = 4
ff_hidden_size = 512
num_layers = 3
dropout = 0.1

# Create transformer encoder
encoder = TransformerECG(embed_size, ecg_length, ecg_channel, num_heads, ff_hidden_size, num_layers, dropout)


In [7]:
for name, param in encoder.named_parameters():
    if "weight" in name:  # Filter only weight matrices
        print(f"Layer: {name} | Matrix size: {param.size()}")

total_params = sum(p.numel() for p in encoder.parameters())
print(f"Total number of parameters in the model: {total_params}")

Layer: embedding.weight | Matrix size: torch.Size([256, 12])
Layer: layers.0.self_attn.in_proj_weight | Matrix size: torch.Size([768, 256])
Layer: layers.0.self_attn.out_proj.weight | Matrix size: torch.Size([256, 256])
Layer: layers.0.feed_forward.fc1.weight | Matrix size: torch.Size([512, 256])
Layer: layers.0.feed_forward.fc2.weight | Matrix size: torch.Size([256, 512])
Layer: layers.0.layer_norm1.weight | Matrix size: torch.Size([256])
Layer: layers.0.layer_norm2.weight | Matrix size: torch.Size([256])
Layer: layers.1.self_attn.in_proj_weight | Matrix size: torch.Size([768, 256])
Layer: layers.1.self_attn.out_proj.weight | Matrix size: torch.Size([256, 256])
Layer: layers.1.feed_forward.fc1.weight | Matrix size: torch.Size([512, 256])
Layer: layers.1.feed_forward.fc2.weight | Matrix size: torch.Size([256, 512])
Layer: layers.1.layer_norm1.weight | Matrix size: torch.Size([256])
Layer: layers.1.layer_norm2.weight | Matrix size: torch.Size([256])
Layer: layers.2.self_attn.in_proj_wei

In [ ]:
encoder(x).shape

torch.Size([4, 10])

In [4]:
model_res(x).shape

torch.Size([4, 10])

In [ ]:

from dataset.mimic_iv_ecg_dataset import DictDataset 
from torch.utils.data import DataLoader 

normal_dataset = DictDataset(path='./prerequisites/clf_data/mimic_vae_clf_{af}_train_cmp.pt') 
dataloader = DataLoader(normal_dataset, 16)

normal_cnt = 0
abnormal_cnt = 0 
for _, y in normal_dataset: 
    if y['label'] == 1:
        normal_cnt += 1
    else:
        abnormal_cnt += 1


print(normal_cnt, abnormal_cnt, normal_cnt / abnormal_cnt) 

25816 24424 1.0569931215198165


        Normal   Abnormal   Ratio       cmp    Ratio
train   25816       14184   1.82      22376     1.15
valid   3207        1793    1.78
test    3255        1745    1.86

        Non-AF        AF    Ratio
train   35791       4209    8.50      34929     1.02
valid   4380        620     7.06
test    4501        499     9.02


        Non-AF       PVC    Ratio
train   37428       2572   14.55      38412     0.97 
valid   4707        293    16.06
test    4716        284    16.60

In [2]:
for (X, y) in dataloader: 
    print(y['label']) 
    break

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [3]:
import numpy as np 
from sklearn.metrics import f1_score 

a = np.array([1, 0, 1])
b = np.array([1, 1, 1]) 

f1_score(a, b, average='weighted')

0.5333333333333333

In [3]:
# Combine Dataset 
import numpy as np 
import torch 

exp_type = 'normal'
ori_dict_path = f'./prerequisites/clf_data/mimic_vae_clf_{exp_type}_train.pt'
ori_dict = torch.load(ori_dict_path) 
print(len(ori_dict)) 
comp_key_offset = max(ori_dict.keys()) + 1

comp_path = f'./prerequisites/clf_data/completion_{exp_type}.npy'
comp_matrix = np.load(comp_path) 
print(comp_matrix.shape) 

for idx in range(comp_matrix.shape[0]): 
    new_key = comp_key_offset + idx 
    ecg = comp_matrix[idx] 
    ecg = torch.from_numpy(ecg)

    value = { 'label': {'label': 0, 'text': 'cmp'}, 
                 'data': ecg}
    ori_dict[new_key] = value 

print(len(ori_dict)) 
torch.save(ori_dict, f'./prerequisites/clf_data/mimic_vae_clf_{exp_type}_train_cmp.pt')

40000
(10240, 4, 128)
50240


In [4]:
import numpy as np 

def weighted_f1_score_from_confusion_matrix(cm):
    # Calculate precision and recall
    # cm: pred: 0   1
    # true  0   tn  fp
    #       1   fn  tp
    tp = cm[1][1]; fp = cm[0][1]; fn = cm[1][0]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Calculate F1 score
    f1_score_1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    tp = cm[0][0]; fp = cm[1][0]; fn = cm[0][1]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    f1_score_0 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    num_0 = sum(cm[0]); num_1 = sum(cm[1]) 

    weighted_f1 = (num_0 * f1_score_0 + num_1 * f1_score_1) / (num_0 + num_1) 

    return f1_score_0, f1_score_1, weighted_f1 

exp_type = 'normal'
idx = 3
model_type = 'ResNet'
cm_1 = np.load(f'./checkpoints/clf_{exp_type}_{model_type}_{idx}/cm.npy')
weighted_f1_score_from_confusion_matrix(cm_1) 

(0.6718036529680367, 0.8229679802955664, 0.7702116300582584)